# MYSQL, LangCahin, Tour DB 이용한 RAG
1. MySQL -> 관광 정보 불러오기
2. LangChain으로 텍스트 쪼개기 + 벡터화 + Chroma 저장
3. 사용자 질문 입력 -> 유사한 관광 정보 검색
4. LangChain + Gemini 로 답변 생성


In [ ]:
%pip install mysql-connector-python sqlalchemy
%pip install langchain langchain-google-genai chromadb python-dotenv

In [ ]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

#langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

load_dotenv()

# DB 연결 위한 SQLAlchemy 엔진 만들기
# engine -> MySQL과 연결된 가상 연결 객체
engine = create_engine(os.getenv("DB_URL"))

In [ ]:
# [TABLE] Content, PET_TOUR_INFO 조인, 데이터 불러오기
with engine.connect() as conn:
    result = conn.execute(text(
        """
        SELECT
            c.contentid,
            c.title,
            c.overview,
            c.addr1,
            c.addr2,
            pt.rela_acdnt_risk_mtr,
            pt.acmpy_type_cd,
            pt.rela_poses_fclty,
            pt.etc_acmpy_info,
            pt.acmpy_need_mtr
        FROM Content c
        JOIN Pet_Tour_Info pt ON c.contentid=pt.contentid
"""))
    
# 쿼리 결과를 텍스트로 조합하기
data = []
for row in result:
    text_parts = [
        f"[이름] {row.title}",
        f"[설명] {row.overview}",
        f"[주소] {row.addr1 or ''} {row.addr2 or ''}",
    ]
    
    # 반려동물 관련 정보 추가
    if row.acmpy_type_cd:
        text_parts.append(f"[동반유형] {row.acmpy_type_cd}")
    if row.rela_acdnt_risk_mtr:
        text_parts.append(f"[위험요소] {row.rela_acdnt_risk_mtr}")
    if row.rela_poses_fclty:
        text_parts.append(f"[구비시설] {row.rela_poses_fclty}")
    if row.etc_acmpy_info:
        text_parts.append(f"[기타정보] {row.etc_acmpy_info}")
    if row.acmpy_need_mtr:
        text_parts.append(f"[유의사항] {row.acmpy_need_mtr}")
        
    combined_text=" ".join(text_parts)
    data.append(combined_text.strip())
    
# print(data[:3])

In [10]:
# 텍스트 청킹
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# splitter 이용하여 자르기
splitter = RecursiveCharacterTextSplitter(
    # 청크 사이즈 300
    chunk_size=300, 
    # 겹치는 글자수
    chunk_overlap=50
)

# 청크 나누기
docs = splitter.create_documents(data)

In [27]:
# 임베딩(벡터화)
# text-multilingual-embedding-002 모델 사용
load_dotenv()
embedding = GoogleGenerativeAIEmbeddings(
    model='models/embedding-001'
)

In [ ]:
# Chroma Vector DB에 벡터화 내용 저장
# (테스트용) 로컬에 저장해서 다음에 사용할 수 있게 -> 추후 Qdrant로 전환

from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    docs,
    embedding=embedding,
    persist_directory="./tour_pet"        
)

vectorstore.persist()

In [ ]:
# retriever > 질문에 가까운 문장 찾기
retriever = vectorstore.as_retriever()

query = "강아지랑 놀러가기 좋은 서울 공원 있어?"
matched_docs = retriever.get_relevant_documents(query)

for i, doc in enumerate(matched_docs[:3], 1):
    print(f"{i}: \n{doc.page_content}")

In [ ]:
# from langchain.chains import RetrievalQA
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(
#       model="gemini-1.5-pro")

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,   # 기존 Chroma retriever 사용
#     return_source_documents=True,  # 답변과 함께 근거 문서도 반환
# )

# query = "강아지 쫑쫑이랑 같이 갈 수 있는 인천에 있는 반려동물 동반 가능한 공원을 알려줘. 관광지 이름도 같이 알려줘."

# result = qa_chain.invoke({"query": query})
# print(result["result"])

In [ ]:
filtered_docs = [doc for doc in docs if "강원" in doc.page_content]
print(f"강원 청크: {len(filtered_docs)}")

from langchain.vectorstores import Chroma

filtered_vectorstore = Chroma.from_documents(
    filtered_docs,
    embedding=embedding
)
filtered_retriever = filtered_vectorstore.as_retriever()

In [76]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
# from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini LLM 설정
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro"
)


In [79]:

# 유도형 프롬프트
custom_prompt = PromptTemplate.from_template(
    """
    다음 정보를 참고해주세요
    {context}
    
    사용자의 질문에 따라 적절한 장소를 다음 형식으로 추천해주세요:
    - 장소명: OOO
    - 설명: 간단한 설명
    - 주소: 가능하면 포함
    - 반려동물 관련 정보 : 최대한 포함

    질문: {question}
    """
)


qa_chain_filtered = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=filtered_retriever,
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True,
    input_key="question"
    
)

query = "말티즈즈 쫑쫑이랑 같이 갈 수 있는 강원도에 있는 반려동물 동반 가능한 펜션, 숙소, 호텔, 모텔을을 알려줘. 관광지 이름도 같이 알려줘."
result = qa_chain_filtered.invoke({"question": query})

print("🐶: 맞춤 장소를 추천해드릴게요!\n")
print(result["result"])

🐶: 맞춤 장소를 추천해드릴게요!

- 장소명: 휘닉스 평창 콘도
- 설명: 다양한 부대시설(수영장, 볼링장, 당구장, 마사지, 편의점 등)을 갖춘 고급 콘도. 반려동물 동반 가능 여부는 명시되지 않았으므로, 콘도에 직접 문의 필요.
- 주소: 강원특별자치도 평창군 봉평면 태기로
- 반려동물 관련 정보:  문의 필요

- 장소명: 무릉도원면 캠핑장 (이름 불명)
- 설명: 수영장, 매점, 미니게임장 등 부대시설을 갖춘 캠핑장. 일부 구역 반려동물 동반 가능. 주변에 고씨동굴, 선돌, 한반도지형 등 관광지 위치.
- 주소: 강원특별자치도 영월군 무릉도원면 도원운학로 475-10
- 반려동물 관련 정보: 일부구역 동반 가능

- 장소명: 법흥계곡 캠핑장 (이름 불명)
- 설명: 캠핑 장비 대여 가능, 365일 운영, 반려견 입장 가능. 주류, 음료, 식품 등 판매하는 매점 있음.
- 주소: 강원특별자치도 영월군 무릉도원면 무릉법흥로 968-12
- 반려동물 관련 정보: 반려견 동반 가능


위 세 곳 외에 질문에 제시된 정보에는 반려동물 동반 가능한 펜션, 숙소, 호텔, 모텔 정보가 없습니다.  평창/영월 지역의 반려동물 동반 숙소는 추가 검색을 통해 찾아보시는 것을 권장합니다.


**추가적으로, 평창 관광 정보:**

허브나라, 월정사 전나무숲길, 효석달빛언덕, 대관령양떼목장, 오대산, 발왕산케이블카 등이 있으며, 평창 관광택시를 이용하여 편리하게 여행 가능합니다. (단, 반려동물 동반 가능 여부는 택시 기사님께 확인 필요)
